In [1]:
import time
import json
import asyncio
import nest_asyncio

from tqdm import tqdm
import concurrent.futures
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from scrapegraphai.graphs import SmartScraperGraph
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

nest_asyncio.apply()

In [11]:
driver = webdriver.Chrome()
options = webdriver.ChromeOptions()

prefs = {
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': True
}

options.add_experimental_option('prefs', prefs)
options.add_argument('--start-maximized')
driver = webdriver.Chrome(options)

url = "https://www.hotcourses.vn/study/international-scholarships/all/postgraduate/computer-and-mathematical-sciences/cid/3/e/scholarship.html"
driver.get(url)

hrefs = []
num_pages = 1
for i in tqdm(range(num_pages)):
    soup = BeautifulSoup(driver.page_source, "html.parser")
    scholars_divs = soup.find("div", id = "middleContent").find("div")
    schorlar_divs = scholars_divs.find_all("div")
    for scholar_div in schorlar_divs:
        a_tag = scholar_div.find("a")
        if not a_tag:
            continue
            
        else:
            href = a_tag["href"]
            if href != "javascript:void(0);":
                hrefs.append(href)
                break

    pager = soup.find("ul", class_ = "pager")
    next_pager = pager.find("li", class_ = "pager__item pager__item--next")
    next_button = next_pager.find("a")

    if next_button:
        next_href = next_button["href"]
        driver.get(next_href)
        time.sleep(3)

def crawl_scholarship_description_v1(div):
    for tag in div.find_all(True):
            if tag.name != 'a':
                tag.unwrap()

    for a in div.find_all('a'):
        href = a.get('href')
        a.attrs = {'href': href} if href else {}

    scholar_div = div.prettify(formatter="minimal")
    return scholar_div


driver.get("https://www.hotcourses.vn/study/uk/school-college-university/university-of-birmingham/5666/international.html")
soup = BeautifulSoup(driver.page_source, "html.parser")

nav_div = driver.find_element(By.CLASS_NAME, "chuni_lft.chcol-2")
ul_element = nav_div.find_element(By.TAG_NAME, "ul")

print(nav_div)
wait = WebDriverWait(driver, 10)

scholarship_tab = wait.until(EC.element_to_be_clickable((By.ID, "menu-nav-tab-2")))
requirements_tab = wait.until(EC.element_to_be_clickable((By.ID, "menu-nav-tab-3")))

driver.execute_script("arguments[0].scrollIntoView();", scholarship_tab)
driver.execute_script("arguments[0].click();", scholarship_tab)
soup = BeautifulSoup(driver.page_source, "html.parser")
scholarship_div = soup.find("div", class_ = "chfin")
print(crawl_scholarship_description_v1(scholarship_div))

driver.execute_script("arguments[0].click();", requirements_tab)
driver.execute_script("arguments[0].click();", requirements_tab)
soup = BeautifulSoup(driver.page_source, "html.parser")
scholarship_div = soup.find("div", class_ = "chfin")
print(crawl_scholarship_description_v1(scholarship_div))

100%|██████████| 1/1 [00:08<00:00,  8.10s/it]


<selenium.webdriver.remote.webelement.WebElement (session="11d33dacbc9197c010bd288e01fa28da", element="f.FF9EF11A10105937FEBBD73FCF11E48F.d.7102A2F2A57BE9CE40DEFBAD5A1E4044.e.1574")>
<div class="chfin" dis-sec-name="finance" ga-sec-name="Dynamic Section 2" id="nav-section-4" sec-id="270" sec-name="Entry requirements" sec-type="content">
 Chi phí và học bổng
 Chí phí
 Học phí trung bình theo năm
 £19,517
 Đại học
 Chi phí chỉ mang tính tham khảo,
 <a href="https://www.hotcoursesinternational.com/study/cpe-track.html?url=https%3A%2F%2Fwww.birmingham.ac.uk%2Findex.aspx%3Futm_source%3DIDP%26utm_medium%3Dreferral%26utm_campaign%3DIDP">
  tốt nhất bạn nên liên hệ nhà trường
 </a>
 để có thông tin chính xác nhất
 £25,693
 Cao học
 Chi phí chỉ mang tính tham khảo,
 <a href="https://www.hotcoursesinternational.com/study/cpe-track.html?url=https%3A%2F%2Fwww.birmingham.ac.uk%2Findex.aspx%3Futm_source%3DIDP%26utm_medium%3Dreferral%26utm_campaign%3DIDP">
  tốt nhất bạn nên liên hệ nhà trường
 </a>
